In [ ]:
import torch
from torch import nn
import torch.optim as optim
from torch.nn import functional as F
from torch.nn.modules.loss import _WeightedLoss
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
# Tabnet 
from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

In [ ]:
MAX_EPOCH = 200


tabnet_params = dict(
    n_d = 32,
    n_a = 32,
    n_steps = 1,
    gamma = 1.3,
    lambda_sparse = 0,
    optimizer_fn = optim.Adam,
    optimizer_params = dict(lr = 2e-2, weight_decay = 1e-5),
    mask_type = "entmax",
    scheduler_params = dict(
        mode = "min", patience = 5, min_lr = 1e-5, factor = 0.9),
    scheduler_fn = ReduceLROnPlateau,
    seed = seed,
    verbose = 10
)


In [ ]:
class LogitsLogLoss(Metric):
    
    #比赛指定评价函数

    def __init__(self):
        self._name = "logits_ll"
    def __call__(self, y_true, y_pred):

        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1 - y_true) * np.log(1 - logits + 1e-15) + y_true * np.log(logits + 1e-15)
        return np.mean(-aux)

In [ ]:
scores_auc_all = []
test_cv_preds = []

NB_SPLITS = 7 # 7


oof_preds = []
oof_targets = []
scores = []
scores_auc = []
SEED = [42,43,44]
# SEED = (42,43,44)  
for s in SEED:
    tabnet_params['seed'] = s
    mskf = MultilabelStratifiedKFold(n_splits = NB_SPLITS, random_state = s, shuffle = True)
    for fold_nb, (train_idx, val_idx) in enumerate(mskf.split(train, train[target_cols])):
        print(b_,"FOLDS: ", r_, fold_nb + 1, y_, 'seed:', tabnet_params['seed'])
        print(g_, '*' * 60, c_)
    
        X_train, y_train = train[feature_cols].values[train_idx, :], train[target_cols].values[train_idx, :]
        X_val, y_val = train[feature_cols].values[val_idx, :], train[target_cols].values[val_idx, :]
    
        model = TabNetRegressor(**tabnet_params)
        
 
        model.fit(
            X_train = X_train,
            y_train = y_train,
            eval_set = [(X_val, y_val)],
            eval_name = ["val"],
            eval_metric = ["logits_ll"],
            max_epochs = MAX_EPOCH,
            patience = 20,
            batch_size = 1024, 
            virtual_batch_size = 32,
            num_workers = 1,
            drop_last = False,
     
            loss_fn = F.binary_cross_entropy_with_logits
        )
        print(y_, '-' * 60)
        

        saving_path_name = 'TabNet_seed_'+str(tabnet_params['seed'])+'_fold_'+str(fold_nb+1)
        saved_filepath = model.save_model(saving_path_name)

        loaded_model =  TabNetRegressor()
        loaded_model.load_model(saved_filepath)


        preds_val = loaded_model.predict(X_val)
        
        #转换为二分类
        preds = 1 / (1 + np.exp(-preds_val))
        score = np.min(model.history["val_logits_ll"])
    
        
        oof_preds.append(preds_val)
        oof_targets.append(y_val)
        scores.append(score)
    
 
        preds_test = loaded_model.predict(x_test[feature_cols].values)
        test_cv_preds.append(1 / (1 + np.exp(-preds_test)))

oof_preds_all = np.concatenate(oof_preds)
oof_targets_all = np.concatenate(oof_targets)
test_preds_all = np.stack(test_cv_preds)